# 기업 시장/재무 데이터 분석 

## 1) 기업 종목 코드 크롤링 

### 1-1) 한국거래소 데이터를 바로 크롤링

밑의 코드를 사용하면 현재 한국거래소에 상장되어있는 기업들의 기업명, 종목코드를 간편하게 수집 가능.  
하지만 이 방법으로는 업종코드를 따로 붙여야함.  

[ 참고: http://excelsior-cjh.tistory.com/109 ] 

In [1]:
# 한국거래소에서 상장기업정보 받기 

import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명','종목코드']]
code_df = code_df.rename(columns={'회사명':'name','종목코드':'code'})

code_df.head()

,name,code
0,CJ,001040
1,HDC,012630
2,HSD엔진,082740
3,KG케미칼,001390
4,LG이노텍,011070


### 1-2) 한국거래소에서 csv 파일 다운로드 

한국거래소에서 다운받은 상장기업정보 csv파일을 dataframe으로 읽어옴.  
업종코드가 붙어있어서 편하지만, 직접 csv파일을 다운받아야하는 번거로움.  

In [2]:
# 한국거래소에서 다운받은 csv파일 데이터프레임으로 저장하기 
#-*- coding: utf-8 -*-

corp_df = pd.read_csv('corp_code2.csv')
corp_df.head()

,code,name,cate_code,cate_name
0,60310,3S,32902,특수 목적용 기계 제조업
1,95570,AJ네트웍스,147603,산업용 기계 및 장비 임대업
2,68400,AJ렌터카,147601,운송장비 임대업
3,6840,AK홀딩스,116409,기타 금융업
4,54620,APS홀딩스,116409,기타 금융업


In [3]:
# 상장된 기업 2208개 확인 

len(corp_df)

2208

In [4]:
# 종목코드 6자리 문자열로 맞추기 
adj_code = []
for i in corp_df.code:
     adj_code.append(str(i).zfill(6))

corp_df.code = adj_code

corp_df.head()

,code,name,cate_code,cate_name
0,060310,3S,32902,특수 목적용 기계 제조업
1,095570,AJ네트웍스,147603,산업용 기계 및 장비 임대업
2,068400,AJ렌터카,147601,운송장비 임대업
3,006840,AK홀딩스,116409,기타 금융업
4,054620,APS홀딩스,116409,기타 금융업


## 2) 기업 재무 데이터 크롤링 

네이버 금융 페이지 웹크롤링 

[ 참고: https://financedata.github.io/posts/naver-finance-finstate-crawling.html ]



In [5]:
# 필요한 모듈 임포트 

import re
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [6]:
# 컬럼명 정리 함수 생성 

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()
        date_str = date_str.replace('/', '-')
    return date_str

In [15]:
def get_finstate_naver(code, fin_type='0', freq_type='Q'):

    # 종목코드에 따른 url 생성 
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                   'cmp_cd=%s&fin_typ=%s&freq_typ=%s'
    url = url_tmpl % (code, fin_type, freq_type)

    # 재무정보 크롤링 
    dfs = pd.read_html(url, encoding="utf-8")
    df = dfs[0]
    df = df.set_index('주요재무정보')
    
    # 컬럼명 정리 
    cols = list(df.columns)
    cols = [get_date_str(cols[i][1]) for i in range(len(cols))]
    df.columns = cols
    df = df.ix[:, :-1]

    # 전치행렬
    dft = df.T
    
    # 컬럼명 정리 2    
    col = list(dft.columns)
    adj_col = [dft.columns[i][0] for i in range(len(col))]
    dft.columns = adj_col

    # date 데이터타입 변경 
    dft.index = pd.to_datetime(dft.index)

    # remove if index is NaT
    dft = dft[pd.notnull(dft.index)]
    dft = dft[pd.notnull(dft.매출액)]
    
    # corp_df에서 코드명 컬럼 가져오기 
    
    dft['code'] = code
    
    
    return dft


get_finstate_naver('054620')

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주),code
2017-06-01,260.0,11.0,11.0,14.0,6825.0,6820.0,5.0,2754.0,804.0,1951.0,...,12981.93,29001.0,1.51,15657.0,2.65,0.0,NaN,0.00,11930928.0,054620
2017-09-01,508.0,30.0,30.0,51.0,41.0,37.0,4.0,2823.0,828.0,1995.0,...,13037.19,278.0,0.38,15986.0,0.74,0.0,NaN,0.00,11930928.0,054620
2017-12-01,299.0,18.0,18.0,13.0,7.0,1.0,6.0,2602.0,700.0,1902.0,...,13037.45,7.0,0.27,15145.0,0.65,0.0,NaN,0.00,11930928.0,054620
2018-03-01,-187.0,-56.0,-56.0,-261.0,-172.0,-164.0,-8.0,2318.0,485.0,1833.0,...,12776.49,-1231.0,0.18,14966.0,0.50,0.0,NaN,-2.72,11930928.0,054620
2018-06-01,69.0,-15.0,-15.0,-11.0,-12.0,-10.0,-2.0,2800.0,470.0,2330.0,...,8375.48,-49.0,-8.20,11856.0,0.62,NaN,NaN,0.00,20394221.0,054620


In [17]:
# 2208개의 상장기업 재무정보 데이터프레임 union join

fin_df = pd.DataFrame()

for i in list(corp_df.code) :
    fin_df = pd.concat([fin_df,get_finstate_naver(i)])

C:\Users\yeseol\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [18]:
len(fin_df)

10776

In [19]:
# csv로 저장 

fin_df.to_csv('fin_df.csv', encoding='utf-8') # 다운은 받아지나 csv 열었을 때 한글컬럼명 깨지는 문제  

## 3) 기업 시장 데이터 크롤링 

### 3-1) 야후 모듈 사용하여 일별 시세 크롤링 

하지만 누락되는 데이터가 보여서 코드만 첨부

[ 참고: https://gomjellie.github.io/%ED%8C%8C%EC%9D%B4%EC%8D%AC/pandas/%EC%A3%BC%EC%8B%9D/2017/06/09/pandas-datareader-stock.html ] 



In [22]:
from pandas_datareader import data
import fix_yahoo_finance as yf
yf.pdr_override()
start_date = '2018-07-01'
tickers = ['067160.KQ', '035420.KS','000660.KS']
afreeca = data.get_data_yahoo(tickers[0], start_date)
naver = data.get_data_yahoo(tickers[1], start_date)
hynix = data.get_data_yahoo(tickers[2], start_date)

naver

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-02,765000.0,765000.0,745000.0,751000.0,751000.0,107918
2018-07-03,759000.0,763000.0,743000.0,747000.0,747000.0,86305
2018-07-04,769000.0,779000.0,760000.0,768000.0,768000.0,179939
2018-07-05,756000.0,769000.0,756000.0,765000.0,765000.0,57319
2018-07-06,755000.0,758000.0,746000.0,749000.0,749000.0,81867
2018-07-09,749000.0,779000.0,748000.0,755000.0,755000.0,94376
2018-07-10,757000.0,768000.0,756000.0,760000.0,760000.0,75129
2018-07-11,763000.0,771000.0,757000.0,769000.0,769000.0,86885
2018-07-12,775000.0,782000.0,764000.0,782000.0,782000.0,95967


### 3-2) 네이버 금융 사이트에서 일별 시세 크롤링 



In [23]:
# 일별시세 크롤링 함수 생성

def get_market_price(code) :
    price_df = pd.DataFrame()
    url_tmpl = 'http://finance.naver.com/item/sise_day.nhn?code=' + str(code)
    for page in range(1,3) : 
        url = url_tmpl + '&page=' + str(page)
        df = pd.read_html(url, header=0)[0]
        price_df = pd.concat([price_df,df])
        price_df = price_df.dropna()
    price_df['code'] = code
    return price_df


get_market_price('054620')
    

,날짜,종가,전일비,시가,고가,저가,거래량,code
1,2018.07.16,5420.0,120.0,5250.0,5460.0,5250.0,101006.0,054620
2,2018.07.13,5300.0,10.0,5380.0,5390.0,5250.0,82662.0,054620
3,2018.07.12,5310.0,10.0,5320.0,5370.0,5270.0,116538.0,054620
4,2018.07.11,5320.0,30.0,5290.0,5400.0,5190.0,143747.0,054620
5,2018.07.10,5290.0,40.0,5300.0,5310.0,5170.0,141518.0,054620
9,2018.07.09,5250.0,310.0,4885.0,5720.0,4885.0,858350.0,054620
10,2018.07.06,4940.0,90.0,4905.0,4945.0,4800.0,41623.0,054620
11,2018.07.05,4850.0,30.0,4775.0,5070.0,4770.0,101324.0,054620
12,2018.07.04,4820.0,40.0,4705.0,4895.0,4705.0,43551.0,054620
13,2018.07.03,4780.0,120.0,4860.0,5020.0,4730.0,105263.0,054620


In [24]:
# 2208개 상장기업 일별시세 합치기 

price_df = pd.DataFrame()

for i in list(corp_df.code) :
    price_df = pd.concat([price_df,get_market_price(i)])

In [25]:
len(price_df)

44032

In [27]:
price_df.to_csv('price_df.csv')